In [5]:
import pandas as pd
import re
import numpy as np

In [7]:
#Cargamos el archivo json
df = pd.read_json('../src/output_steam_games.json', lines=True)

In [8]:
#Limpiamos los datos

#Eliminamos los registros que parecen venir de la concatenación de otra tabla, usando la columna publisher como proxy para identificarlos
df = df.dropna(subset=['publisher'])

#Eliminamos las columnas que no nos interesan
df = df.drop(columns=['publisher', 'url', 'reviews_url', 'discount_price', 'early_access', 'user_id', 'items', 'items_count', 'steam_id' ])

#Llevamos los nombres a minúsculas y quitamos caracteres especiales
df['app_name'] = df['app_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))

#Reemplazamos el &amp; de algunos géneros por &
df['genres'] = df['genres'].apply(lambda x: str(x).replace('&amp;', '&'))

#COnvertimos el valor free to play a 0 y también los nulos
df['price'] = df['price'].apply(lambda x: 0 if isinstance(x, str) else x)
df['price'] = df['price'].apply(lambda x: 0 if pd.isnull(x) else x)

#Eliminamos los registros sin fecha de lanzamiento y extraemos el año, eliminando los registros con fechas inválidas
df = df.dropna(subset=['release_date'])
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date'] = df['release_date'].dt.year
df = df.dropna(subset=['release_date'])

#Quitamos el registro sin id
df.dropna(subset=['id'], inplace=True)

#Imputamos el promedio de metascore a los nulos
df['metascore'] = df['metascore'].apply(lambda x: np.nan if pd.isnull(x) or x == 'NA' else x)
df['metascore'].fillna(df['metascore'].mean(), inplace=True)

#Convertimos los tipos de algunas columnas para evitar errores al inferirlos
df = df.convert_dtypes()
df = df.astype({'genres': str, 'release_date': int, 'tags': str, 'specs': str, 'metascore': float})

In [13]:
#Guardamos el resultado en un archivo parquet
df.to_parquet('../src/cleaned/games.parquet', index=False)